In [81]:
import json
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

### **Download data**

In [82]:
with open('boxplot_results.json', 'r') as file:
    data = json.load(file)

In [83]:
# Initialize an empty list to store data
df_data = []

# Iterate over the keys in the JSON data to extract the values and create rows for the DataFrame
for key, values in data.items():
    parts = key.split('_')
    model = int(parts[1][-1])  # Extract the model number
    enemy = int(parts[2][1])  # Extract the enemy number
    for gain in values:
        df_data.append([model, enemy, gain])

# Create a DataFrame from the list of data
df = pd.DataFrame(df_data, columns=["Model", "Enemy", "Gain"])

### **One-way ANOVA tests for 3 enemies separately**

In [85]:
fitness = df[df['Model'] == 'Fitness']
no_fitness = df[df['Model'] == 'No fitness']

In [92]:
for i in [2, 5, 7]:
  mod = ols('Gain ~ Model', data=df[df['Enemy'] == i]).fit()
  res = sm.stats.anova_lm(mod, typ=2)
  print("-"*17 + " Enemy " + str(i) + " " + "-"*17)
  print(res)
  print("Significant difference: " + str(mod.params.Model.round(4)))

----------------- Enemy 2 -----------------
          sum_sq    df         F    PR(>F)
Model       72.2   1.0  9.309456  0.006875
Residual   139.6  18.0       NaN       NaN
Significant difference: 3.8
----------------- Enemy 5 -----------------
           sum_sq    df         F    PR(>F)
Model       2.592   1.0  0.362416  0.554676
Residual  128.736  18.0       NaN       NaN
Significant difference: -0.72
----------------- Enemy 7 -----------------
            sum_sq    df         F   PR(>F)
Model      115.200   1.0  1.999445  0.17442
Residual  1037.088  18.0       NaN      NaN
Significant difference: 4.8


Three one-way ANOVA tests were also conducted to determine the effect of algorithm choice (with and without fitness sharing) on average gain for each enemy separately. The results indicate that the differences between the two algorithms are statistically significant for enemy 2 (p-value = 0.0069). However, our experiment also showed that we don’t have sufficient evidence to say that there is a statistically significant difference between the two tested algorithms for enemies 5 and 7 (p-values are 0.5547 and 0.1744, respectively).

### **Two-way ANOVA test**

**Null Hypothesis (H0):** There is no significant difference between the groups of the independent variable Model in relation to the dependent variable Gain.

*(any observed variations in the data are purely due to random sampling variability or chance)*


**Alternative Hypothesis (H1):** There is a significant difference between the groups of the independent variable Model in relation to the dependent variable Gain.

*(observed variations in the data are not merely due to random chance but are the result of a genuine effect or relationship)*

In [84]:
formula = 'Gain ~ C(Model) + C(Enemy) + C(Model):C(Enemy)'
model = ols(formula, data=df).fit()
result = sm.stats.anova_lm(model, type=2)

# Print the result
print(result.round(4))

                     df     sum_sq    mean_sq         F  PR(>F)
C(Model)            1.0   103.4907   103.4907    4.2810  0.0433
C(Enemy)            2.0  5261.7973  2630.8987  108.8294  0.0000
C(Model):C(Enemy)   2.0    86.5013    43.2507    1.7891  0.1769
Residual           54.0  1305.4240    24.1745       NaN     NaN


* **Model: 0.0433 < 0.05 => factor Model has a statistically significant effect on Gain / There is significant difference between the groups of the independent variable Model in relation to the dependent variable Gain.**

* Enemy: 0 < 0.05 => factor Enemy has a statistically significant effect on Gain / There is significant difference between the groups of the independent variable Enemy in relation to the dependent variable Gain.

* Model*Enemy: 0.1769 > 0.05 => not statistically significant

**=> the effect of Model is consistent across all Enemies (All group means are equal at each level of the Enemy)**

& the effect of Enemy is consistent across all Models (All group means are equal at each level of the Model)